TEST ML_composer 


In [1]:
import os
from GS_composer import *

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [6]:
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    gpu = physical_devices[0]
    print(gpu)
    assert gpu.device_type == 'GPU'

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [29]:
import os
print(os.getcwd())
#os.chdir("F:/project/sugarcane_disease/")
#os.chdir("O:/project/sugarcane_disease/")
os.getcwd()
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
#user_profile = "C:/Users/uqcche32/OneDrive - The University of Queensland/"
user_profile = "H:/Data/"
#os.chdir("E:/learning resource/OneDrive - The University of Queensland/PhD/HPC_Results/Sugarcane_disease/ML/")
#os.chdir("C:/Users/uqcche32/OneDrive - The University of Queensland/PhD/HPC_Results/Sugarcane_disease/ML/")

H:\PhD_Notebook\Code\ML_composer


In [7]:
tf.config.experimental.get_device_details(gpu)

{'device_name': 'NVIDIA GeForce RTX 4090', 'compute_capability': (8, 9)}

In [30]:
from ClassModel import *
from CustomLayers import *

In [31]:
from Functions import *

In [38]:
class p_args:
    
    def __init__(self):
        
        self.ped = user_profile +"sugarcane_disease/subset/disease_subset"
        self.pheno = user_profile +"sugarcane_disease/subset/disease_subset.phen"
        self.mpheno = 1
        self.index = user_profile +"sugarcane_disease/subset/disease_subset.index"
        self.model = "MultiHead Attention LNN"
        self.annotation =  user_profile +"sugarcane_disease/subset/disease_subset.anno"
        self.trait = "smut"
        self.output = "./Attention_Test_win"
        
        self.load = None
        self.save = True
        self.plot = True
        
        self.round = 1
        self.epoch = 10
        self.rank =False
        self.lr = 0.0001
        self.batch = 32
        self.mean = False
        self.width = 64
        self.depth = 2
        self.quiet = 1
        self.num_heads = 1
        self.residual = True
        self.embedding = 12
        self.data_type = "numerous"
        self.activation = "linear"
        self.classes = 0
        
        
args = p_args()

In [39]:
config = None


In [40]:
if args.output[0] == "/":
    locat = '/' + args.output.strip('/') + '/'
else:
    locat = args.output.strip('/') + '/'
if not os.path.exists(locat):
    os.mkdir(locat)

In [41]:
composer = ML_composer()
composer._model
composer.silence_mode = 2

In [42]:
#composer.get_data(config,args)
composer.args = args

In [43]:
composer.get_data(config,args=args)

1    939
3    940
4    940
2    940
5    943
Name: 2, dtype: int64
Get genotype shape: (4702, 100)
   7   8   9   10  11  12  13  14  15
1   0   2   0   1   2   1   0   1   2
2   1   2   1   1   2   1   0   1   1
3   1   2   0   0   1   0   1   1   2
4   1   2   0   0   1   0   1   1   2
5   0   2   1   1   2   0   2   1   1
6   0   2   0   1   2   1   0   1   2
7   1   2   1   1   2   1   2   1   2
8   0   2   0   1   1   0   1   0   2
9   1   2   1   1   2   1   2   1   2
Running data check
GENO
FAM
PHENO
INDEX
SNPs in annotation file are not ordered by map file
Got LD shape:
(100, 6)


In [44]:
#composer.prepare_model()
#composer._model["INIT_MODEL"].init_model()

In [45]:
args.model

'MultiHead Attention LNN'

In [46]:
index_ref = composer.prepare_cross_validate()

In [47]:
print("Cross-validate: {}".format(index_ref[0]))

Cross-validate: ([2, 3, 4, 5], [1])


In [48]:
train_idx,valid_idx = index_ref[0]

In [49]:
composer.prepare_training(train_idx,valid_idx)
#composer.batchSize = 64

Overall population: 4702
759 individuals need to be removed due to the miss phenotype
Filtered population: 3943
Mean of train phenotype: 4.715129635707253
Use raw phenotype as the target
16    1.0
39    3.0
44    3.0
47    2.0
92    3.0
Name: 2, dtype: float64


In [50]:
#composer._raw_data["GENO"].iloc[:,6:].describe()

In [51]:
composer.compose(train_idx,valid_idx)

USE LocalRealFormer MODEL as training method
Convert data to np.array float32
The transformed SNP shape: (3047, 100, 1)
The transformed SNP shape: (3047,) <dtype: 'float32'>
USE LocalRealFormer MODEL as training method
Convert data to np.array float32
The transformed SNP shape: (896, 100, 1)
The transformed SNP shape: (896,) <dtype: 'float32'>
Train status:
Epochs:  10
Repeat(Round):  1
Got input shape: (100, 1)
(None, 100, 1)
[TensorShape([None, 6, 12])]
[TensorShape([None, 6, 12]), TensorShape([None, 6, 12])]
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer_1 (InputLayer)      [(None, 100, 1)]     0                                            
__________________________________________________________________________________________________
snp_block_layer (SNPBlockLayer) (None, 6, 12)        1200        input_l

H:\miniconda\envs\tf25\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3704: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


96/96 [==============================] - 8s 56ms/step - loss: 7.9345 - acc: 0.0305 - val_loss: 6.0950 - val_acc: 0.0190
Epoch 2/10
96/96 [==============================] - 5s 54ms/step - loss: 5.9170 - acc: 0.0309 - val_loss: 3.9847 - val_acc: 0.0190
Epoch 3/10
96/96 [==============================] - 5s 53ms/step - loss: 5.6346 - acc: 0.0309 - val_loss: 3.9216 - val_acc: 0.0190
Epoch 4/10
96/96 [==============================] - 5s 57ms/step - loss: 5.4629 - acc: 0.0309 - val_loss: 4.3860 - val_acc: 0.0190
Epoch 5/10
96/96 [==============================] - 5s 53ms/step - loss: 5.4444 - acc: 0.0309 - val_loss: 4.5693 - val_acc: 0.0190
Epoch 6/10
96/96 [==============================] - 5s 53ms/step - loss: 5.3979 - acc: 0.0309 - val_loss: 4.2611 - val_acc: 0.0190
Epoch 7/10
96/96 [==============================] - 5s 53ms/step - loss: 5.3388 - acc: 0.0309 - val_loss: 4.0189 - val_acc: 0.0190
Epoch 8/10
96/96 [==============================] - 5s 54ms/step - loss: 5.3409 - acc: 0.0309 

H:\miniconda\envs\tf25\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Model saved.
Resource check:
Total memory: 63.7146 GB
Currently using memory: 2.5968 GB
Ratio of used memory: 16.5000  %
Number of CPU node:  32
Plotting the training process...
H:\PhD_Notebook\Code\ML_composer\Attention_Test_win/smut_MultiHeadAttentionLNN_smut
Plotting loss history...
Plot name:  H:\PhD_Notebook\Code\ML_composer\Attention_Test_win/smut_MultiHeadAttentionLNN_smut/smut_MultiHeadAttentionLNN_1.png
Result:
  Trait      TrainSet ValidSet                  Model  Test_Accuracy  \
0  smut  [2, 3, 4, 5]      [1]  MultiHeadAttentionLNN       0.313108   

   Valid_Accuracy       MSE   Runtime  
0        0.068284  4.070358  0.916667  


In [52]:
composer._model["TRAINED_MODEL"].save("./test_model")

#import tensorflow as tf 

if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

H:\miniconda\envs\tf25\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: ./test_model\assets
Default GPU Device:/device:GPU:0


In [ ]:
1.5//1

In [ ]:
os.getcwd()

In [ ]:
index_ref = composer.prepare_cross_validate()
i = 1
for train_idx,valid_idx in index_ref:
    print("Cross-validate: {}".format(i))
    composer.prepare_training(train_idx,valid_idx)
    composer.compose(train_idx,valid_idx)
    i+=1